## Carregando dados no spark

In [19]:
#Criar pastas no HDFS
#hdfs dfs -mkdir -p  /user/vagrant/dados/si_env-2019/
#hdfs dfs -mkdir -p  /user/vagrant/dados/si_log-2019/
#hdfs dfs -mkdir -p  /user/vagrant/dados/si_bol-2019/

#Copiar arquivos para o HDFS
#hdfs dfs -put -f  /vagrant/TarefaFinalV2/si_env-2019.csv /user/vagrant/dados/si_env-2019/
#hdfs dfs -put -f  /vagrant/TarefaFinalV2/si_bol-2019.csv /user/vagrant/dados/si_bol-2019/
#hdfs dfs -put -f  /vagrant/TarefaFinalV2/si_log-2019.csv /user/vagrant/dados/si_log-2019/

# carregando arquivos do hdfs
df_si_env = spark.read.csv("hdfs://node1:8020//user/vagrant/dados/si_env-2019/si_env-2019.csv", sep=';', header=True, encoding = 'latin1')
df_si_bol = spark.read.csv("hdfs://node1:8020//user/vagrant/dados/si_bol-2019/si_bol-2019.csv", sep=';', header=True, encoding = 'latin1')
df_si_log = spark.read.csv("hdfs://node1:8020//user/vagrant/dados/si_log-2019/si_log-2019.csv", sep=';', header=True, encoding = 'latin1')
df_si_env.show()
df_si_bol.show()
df_si_log.show()

+------------------+-----------------+-----------+--------+--------------+---------------+----+---------------+-------------+-----+----------+---------------------+---------------------+----------------+---------------------+--------------------+--------+----------+
|       num_boletim|data_hora_boletim|n_envolvido|condutor|cod_severidade|desc_severidade|sexo|cinto_seguranca|   Embreagues|Idade|nascimento|categoria_habilitacao|descricao_habilitacao|declaracao_obito|cod_severidade_antiga|     especie_veiculo|pedestre|passageiro|
+------------------+-----------------+-----------+--------+--------------+---------------+----+---------------+-------------+-----+----------+---------------------+---------------------+----------------+---------------------+--------------------+--------+----------+
|2019-007782522-001| 18/02/2019 19:24|          2|       S|             1|NAO FATAL      |   M|            SIM|          NÃO|   32|02/05/1986|                  AB | HABILITADO NAS CA...|             

+------------------+----------------+------------+--------------------+---------------+-------------+---------------+--------------------+------------------------+------------------------+---------+--------------------+-----------+---------------------+---------+-----------------+
|       num_boletim|    data_boletim|N°_municipio|      nome_municipio|seq_logradouros|N°_logradouro|tipo_logradouro|     nome_logradouro|tipo_logradouro_anterior|nome_logradouro_anterior|N°_bairro|         nome_bairro|tipo_bairro|descricao_tipo_bairro|N°_imovel|N°_imovel_proximo|
+------------------+----------------+------------+--------------------+---------------+-------------+---------------+--------------------+------------------------+------------------------+---------+--------------------+-----------+---------------------+---------+-----------------+
|2019-049379035-001|09/10/2019 06:46|           1|BELO HORIZONTE   ...|              1|       117712|            RUA|JOAO ARANTES     ...|                

## Selecionando colunas

In [21]:
#Join entre todos os objetos
dfjoin = df_si_env.join(df_si_bol.join(df_si_log, on='num_boletim', how='left'), on='num_boletim', how='left')

#Importa a function
import pyspark.sql.functions as F

#Exibe o Join 
dfjoin.show()

+------------------+-----------------+-----------+--------+--------------+---------------+----+---------------+-------------+-----+----------+---------------------+---------------------+----------------+---------------------+--------------------+--------+----------+-----------------+----------------+-------------+--------------------+---------+---------------+-------------+---------------+------------+---------------+---------------+----------------+--------------------+-------------+-------------+--------------+--------------------+---------+-------------+-------------------+----------------+--------------------+----------------+------------+--------------------+---------------+-------------+---------------+--------------------+------------------------+------------------------+---------+--------------------+-----------+---------------------+---------+-----------------+
|       num_boletim|data_hora_boletim|n_envolvido|condutor|cod_severidade|desc_severidade|sexo|cinto_seguranca|   Emb

In [22]:
# 1) Total de acidentes com vítima por bairro em acidentes com embriaguez;

#Efetua o filtro e o agrupamento.
dfFiltroAgrupado = dfjoin.filter((dfjoin.Embreagues == 'SIM') & ((dfjoin.DESC_TIPO_ACIDENTE.like('%COM VITIMA%')) | dfjoin.DESC_TIPO_ACIDENTE.like('%C/ VITIMA%'))).groupby('nome_bairro')

#Calcula o resultado
dfResultado = dfFiltroAgrupado.agg(F.count('*').alias('TotalAcidenteComVitima'))

#xibe o Resultado
dfResultado.show()

+--------------------+----------------------+
|         nome_bairro|TotalAcidenteComVitima|
+--------------------+----------------------+
|SANTO ANTONIO    ...|                     2|
|PAQUETA          ...|                     1|
|LOURDES          ...|                    11|
|SION             ...|                     1|
|HELIOPOLIS       ...|                     8|
|SILVEIRA         ...|                     5|
|DOM SILVERIO     ...|                     1|
|CAICARAS         ...|                     1|
|IPIRANGA         ...|                     3|
|SANTO ANDRE      ...|                     2|
|SANTA HELENA     ...|                     2|
|JARDIM LEBLON    ...|                     1|
|SAO GABRIEL      ...|                     3|
|JOAO PINHEIRO    ...|                     1|
|SAO GERALDO      ...|                     2|
|JARDIM AMERICA   ...|                     4|
|LETICIA          ...|                     3|
|SANTA BRANCA     ...|                     4|
|SANTA TEREZA     ...|            

In [23]:
#2) Total de acidentes por tipo de pavimento e condição do tempo;

#Executa o Groupby, calcula e já exibe
dfjoin.groupby('PAVIMENTO', 'DESC_TEMPO').agg(F.count('*').alias('TotalAcidentes')).show()


+---------------+---------------+--------------+
|      PAVIMENTO|     DESC_TEMPO|TotalAcidentes|
+---------------+---------------+--------------+
|CALCAMENTO     |CHUVA          |            14|
|CALCAMENTO     |NAO INFORMADO  |             4|
|NAO INFORMADO  |NAO INFORMADO  |         18884|
|NAO INFORMADO  |NUBLADO        |             4|
|CONCRETO       |NUBLADO        |            52|
|ASFALTO        |BOM            |         16535|
|NAO INFORMADO  |CHUVA          |            16|
|TERRA          |BOM            |             3|
|ASFALTO        |NEBLINA        |            19|
|CONCRETO       |BOM            |           129|
|CALCAMENTO     |NUBLADO        |             7|
|ASFALTO        |CHUVA          |          1358|
|CALCAMENTO     |BOM            |           109|
|NAO INFORMADO  |BOM            |           124|
|TERRA          |NUBLADO        |             2|
|CONCRETO       |CHUVA          |             6|
|ASFALTO        |NUBLADO        |           825|
|ASFALTO        |NAO

In [24]:
#3) Total de pessoas acidentadas por tipo de veiculo e tipo de pavimentação;

#Efetua o agrupamento.
dfAgrupado = dfjoin.groupby('PAVIMENTO', 'especie_veiculo')

#Calcula o resultado e já exibe
dfAgrupado.agg(F.sum('n_envolvido').alias('TotalPessoasEnvolvidas'),F.count('*').alias('TotalAcidente')).show()

+---------------+--------------------+----------------------+-------------+
|      PAVIMENTO|     especie_veiculo|TotalPessoasEnvolvidas|TotalAcidente|
+---------------+--------------------+----------------------+-------------+
|CALCAMENTO     |CAMINHAO         ...|                   3.0|            1|
|CONCRETO       |MOTONETA         ...|                   4.0|            3|
|NAO INFORMADO  |TRACAO           ...|                   2.0|            1|
|CALCAMENTO     |                 ...|                  15.0|            7|
|NAO INFORMADO  |REBOQUE E SEMI-RE...|                   6.0|            4|
|NAO INFORMADO  |AUTOMOVEL        ...|               13659.0|         7777|
|CALCAMENTO     |CAMINHONETE      ...|                   2.0|            2|
|NAO INFORMADO  |CICLOMOTOR       ...|                   4.0|            4|
|NAO INFORMADO  |MOTONETA         ...|                 146.0|          107|
|ASFALTO        |MOTOCICLETA      ...|               11479.0|         6847|
|ASFALTO    

In [25]:
#4) Média de idade dos condutores por tipo de veículo e tipo de acidente;

#Efetua o filtro e o agrupamento.
dfAgrupado = dfjoin.filter(dfjoin.condutor == 'S').groupby('especie_veiculo', 'TIPO_ACIDENTE', 'DESC_TIPO_ACIDENTE')

#Efetua calculo
dfAgrupado.agg(F.avg('Idade').alias('media_idade')).show()

+--------------------+-------------+--------------------+-----------------+
|     especie_veiculo|TIPO_ACIDENTE|  DESC_TIPO_ACIDENTE|      media_idade|
+--------------------+-------------+--------------------+-----------------+
|CAMIONETA        ...|       H01002|ABALROAMENTO COM ...|38.40127388535032|
|TRICICLO         ...|       H01002|ABALROAMENTO COM ...|             14.5|
|CAMINHONETE      ...|       H06001|ATROPELAMENTO DE ...|             24.0|
|MOTOCICLETA      ...|       H01002|ABALROAMENTO COM ...|30.32597437994004|
|MOTONETA         ...|       H07002|ATROPELAMENTO DE ...|             24.0|
|TAXI             ...|       H01002|ABALROAMENTO COM ...|              0.0|
|MOTOCICLETA      ...|       H02001|CAPOTAMENTO/TOMBA...|             19.0|
|BICICLETA        ...|       H07002|ATROPELAMENTO DE ...|             46.0|
|MICROONIBUS      ...|       H04000|QUEDA DE PESSOA D...|             45.8|
|TRICICLO         ...|       H08002|CHOQUE MECANICO C...|             46.0|
|NAO INFORMA

In [26]:
#5) Média de idade dos condutores por indicativo de embriaguez;

#Efetua o filtro e o agrupamento.
dfAgrupado = dfjoin.filter(dfjoin.condutor == 'S').groupby('Embreagues')
#Efetua calculo
dfAgrupado.agg(F.avg('Idade').alias('media_idade')).show()

+-------------+------------------+
|   Embreagues|       media_idade|
+-------------+------------------+
|          NÃO| 36.90902907668764|
|          SIM|  36.5187969924812|
|NÃO INFORMADO|0.9450499545867393|
+-------------+------------------+

